#2 SP500

In [2]:
"""
@author:Zhirong Huang Practice Q2 - S&P
"""

from gurobipy import GRB
import gurobipy as gb
import pandas as pd



In [10]:
from gurobipy import Model, GRB
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/ZorroHZR/MMAI-5000/main/omis%206000%20mid/sp500_data.csv')

# Initialize the model
m = Model('investment_portfolio')

# Decision variables: Amount to invest in each stock
investments = m.addVars(df.index, name='investment', lb=0, ub=600000)

# Objective: Maximize expected return
# Using 'PercentReturn' as the expected return
m.setObjective(sum(investments[i] * df.loc[i, 'PercentReturn'] for i in df.index), GRB.MAXIMIZE)

# Constraint: Total investment is $10 million
m.addConstr(sum(investments[i] for i in df.index) == 10000000, 'total_investment')

# Adjusted constraints using 'GICS Sector' and 'Location of Headquarters'
# Telecommunications sector limit
telecom_indices = df[df['GICS Sector'] == 'Telecommunications Services'].index
m.addConstr(sum(investments[i] for i in telecom_indices) <= 500000, 'telecom_limit')

# IT sector investment should be at least 75% of telecommunications sector investment
it_indices = df[df['GICS Sector'] == 'Information Technology'].index
m.addConstr(sum(investments[i] for i in it_indices) >= 0.75 * sum(investments[i] for i in telecom_indices), 'it_telecom_relation')

# Difference between Consumer Discretionary and Consumer Staples sectors
cd_indices = df[df['GICS Sector'] == 'Consumer Discretionary'].index
cs_indices = df[df['GICS Sector'] == 'Consumer Staples'].index
m.addConstr(sum(investments[i] for i in cd_indices) - sum(investments[i] for i in cs_indices) <= 200000, 'consumer_difference_upper')
m.addConstr(sum(investments[i] for i in cs_indices) - sum(investments[i] for i in cd_indices) <= 200000, 'consumer_difference_lower')

# Energy sector minimum investment
energy_indices = df[df['GICS Sector'] == 'Energy'].index
m.addConstr(sum(investments[i] for i in energy_indices) >= 1000000, 'energy_minimum')

# New York-based companies minimum investment
ny_indices = df[df['Location of Headquarters'].str.contains('New York, New York')].index
m.addConstr(sum(investments[i] for i in ny_indices) >= 300000, 'ny_minimum')

# Solve the model
m.optimize()

# Retrieve the optimal investment amounts
if m.status == GRB.OPTIMAL:
    investment_values = m.getAttr('x', investments)
    for i in df.index:
        print(f"Invest ${investment_values[i]:.2f} in stock {df.loc[i, 'Company']}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 7 rows, 67 columns and 129 nonzeros
Model fingerprint: 0xccecd0b8
Coefficient statistics:
  Matrix range     [8e-01, 1e+00]
  Objective range  [1e-01, 1e+01]
  Bounds range     [6e+05, 6e+05]
  RHS range        [2e+05, 1e+07]
Presolve removed 1 rows and 16 columns
Presolve time: 0.00s
Presolved: 6 rows, 52 columns, 92 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4560000e+08   3.474937e+06   0.000000e+00      0s
       6    5.1346000e+07   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.134600000e+07
Invest $600000.00 in stock Ameren Corp
Invest $0.00 in stock American Express Co
Invest $0.00 in stock AT&T Inc
Inve

In [11]:
# Counting companies headquartered in NYC
nyc_headquarters_count = df[df['Location of Headquarters'].str.contains('New York, New York')].shape[0]
print(f"Number of companies headquartered in New York City: {nyc_headquarters_count}")


Number of companies headquartered in New York City: 6


In [12]:
# Constraint for maximum investment in any individual stock
for i in df.index:
    print(f"x_{i} <= 600000")


x_0 <= 600000
x_1 <= 600000
x_2 <= 600000
x_3 <= 600000
x_4 <= 600000
x_5 <= 600000
x_6 <= 600000
x_7 <= 600000
x_8 <= 600000
x_9 <= 600000
x_10 <= 600000
x_11 <= 600000
x_12 <= 600000
x_13 <= 600000
x_14 <= 600000
x_15 <= 600000
x_16 <= 600000
x_17 <= 600000
x_18 <= 600000
x_19 <= 600000
x_20 <= 600000
x_21 <= 600000
x_22 <= 600000
x_23 <= 600000
x_24 <= 600000
x_25 <= 600000
x_26 <= 600000
x_27 <= 600000
x_28 <= 600000
x_29 <= 600000
x_30 <= 600000
x_31 <= 600000
x_32 <= 600000
x_33 <= 600000
x_34 <= 600000
x_35 <= 600000
x_36 <= 600000
x_37 <= 600000
x_38 <= 600000
x_39 <= 600000
x_40 <= 600000
x_41 <= 600000
x_42 <= 600000
x_43 <= 600000
x_44 <= 600000
x_45 <= 600000
x_46 <= 600000
x_47 <= 600000
x_48 <= 600000
x_49 <= 600000
x_50 <= 600000
x_51 <= 600000
x_52 <= 600000
x_53 <= 600000
x_54 <= 600000
x_55 <= 600000
x_56 <= 600000
x_57 <= 600000
x_58 <= 600000
x_59 <= 600000
x_60 <= 600000
x_61 <= 600000
x_62 <= 600000
x_63 <= 600000
x_64 <= 600000
x_65 <= 600000
x_66 <= 600000


In [13]:
# Calculate total investment in companies headquartered in NYC
nyc_total_investment = sum(investment_values[i] for i in ny_indices)
print(f"Total investment in companies headquartered in NYC: ${nyc_total_investment:.2f}")


Total investment in companies headquartered in NYC: $600000.00


In [14]:
# Calculate the total expected return
total_expected_return = sum(investment_values[i] * df.loc[i, 'PercentReturn'] for i in df.index)
print(f"Total expected return after 1 year: ${total_expected_return:.2f}")


Total expected return after 1 year: $51346000.00


How many companies are headquartered in New York City?
To find the number of companies headquartered in New York City, we can filter the dataset based on the 'Location of Headquarters' column.

How many decision variables are there?
There are 67 decision variables, as each company represents a decision variable for the investment amount.

What does a decision variable represent?
A decision variable represents the amount of money to be invested in the corresponding stock.

Constraint for individual stock investment:
The constraint associated with the restriction that at most $600,000 can be invested in any individual stock is:
�
�
≤
600
,
000
∀
�
∈
{
1
,
2
,
…
,
67
}
x 
i
​
 ≤600,000∀i∈{1,2,…,67}

How much is invested in companies headquartered in NYC?
To calculate the total investment in NYC-headquartered companies, sum the investment values for companies with their headquarters in New York, New York.

What is the optimal expected return of the portfolio after 1 year?
The optimal expected return is given by the objective value, which is $51,346,000.

Regarding the Energy sector investment reduction query:
The shadow price for the Energy sector constraint is -1.31%. This indicates the rate at which the objective value (expected return) would decrease if the right-hand side of the Energy sector constraint (minimum investment requirement) is increased by one unit (here, one dollar). Since the shadow price is negative, reducing the investment in the Energy sector (loosening the constraint) would increase the expected return. Therefore, it could be worth inquiring about reducing the investment in the Energy sector.

Impact of the new return for Coca-Cola Enterprises:
If the return for Coca-Cola Enterprises increases to 3.00% and the allowable increase for its return coefficient is 2.4%, the new return is within the allowable increase range. However, without re-solving the model, we cannot definitively say whether the investment in Coca-Cola Enterprises should be increased. The decision would depend on how this change affects the overall portfolio optimization, considering all constraints and the interplay of returns across different investments. Given that the change is within the allowable range, it's unlikely to impact the current optimal solution significantly, but a precise answer would require re-solving the model with the updated return.